# GPT API를 통한 데이터 정제 예시

참조 자료: [OpenAI의 공식 문서](https://platform.openai.com/docs/guides/gpt)

In [ ]:
# !pip install openai

**Key 생성 방법**

- OpenAI Platform의 API Key 화면에 접속: https://platform.openai.com/api-keys
- "Create new secret key" 클릭해서 생성
- 생성한 API 키는 다시 확인이 되지 않으므로 창을 닫기 전에 복사, 저장해 둘것
- 타인에게 절대로 노출 시키지 말것 $\rightarrow$ 쓴 만큼 돈 나감

In [ ]:
# 4.1
from openai import OpenAI
client = OpenAI(api_key="별도 제공 예정")

## 주소 분류하기

In [ ]:
# 4.2 [TASK] "address_data.csv"를 불러옵니다


In [ ]:
# 4.3 사전에 정의된 함수
def get_region(location):
    PROMPT = f"""
    {location}: 이 주소를 다음 중 하나로 분류하면? 부연 없이 분류 결과만 말해줘
    서울, 부산, 대구, 인천, 광주, 대전, 울산, 세종, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주
    """
    response = client.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": PROMPT}],
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
get_region("부천시청역")

_**주의: 이 과정은 모든 분들이 한꺼번에 실행할 경우 오류 가능성이 높습니다.**_

여기서는 첫 5줄만 시험 삼아 해보도록 합니다. 만약 에러가 날 경우, 나중에 다시 실행해보시기를 권장합니다. 실습 후 업무 시 활용 방안을 위해 아래 2개 링크를 첨부합니다.

GPT 3.5 기준으로 아래 59개 주소를 처리하기 위한 비용은 약 $0.01(10~15원)입니다.

유사하게 사용 시, 민감한 정보는 전송되지 않도록 유의해주세요.

In [ ]:
# 4.4 OpenAI API 적용
address_data_test = address_data.head()
address_data_test["region"] = address_data_test["location"].apply(get_region)
address_data_test

## 고장 내역 데이터 처리하기

In [ ]:
# 4.5 [TASK] "ng_data.csv"를 불러옵니다


In [ ]:
# 4.6
def categorize_ng(text):
    PROMPT = f"""
    '{text}'
    가상의 고장 내역이야.
    부품불량, 오염, 장비고장 중 하나로 분류해줘. 부연 없이 정해진 분류만 답해줘.
    """
    response = client.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=[{"role": "user", "content": PROMPT}],
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
# 4.7
ng_test = ng_data["ng"][1]
print(ng_test)
categorize_ng(ng_test)

In [ ]:
# 4.8
ng_data_test = ng_data.head()
ng_data_test["category"] = ng_data_test["ng"].apply(categorize_ng)
ng_data_test